<a href="https://colab.research.google.com/github/rezve66/Genetic-Diversity/blob/main/Final_SSR_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ============================================
# 📊 SSR Genetic Diversity and Differentiation Analysis
# Google Colab Ready Script
# ============================================

# Import necessary libraries
import pandas as pd
import numpy as np
from google.colab import files

# ----------------------------------------------------------
# 🧬 Core Genetic Diversity Functions
# ----------------------------------------------------------

# Major Allele Frequency (MAF)
def major_allele_frequency(marker_data):
    unique, counts = np.unique(marker_data, return_counts=True)
    major_allele_freq = max(counts) / len(marker_data)
    return major_allele_freq

# Number of Alleles (Na)
def number_of_alleles(marker_data):
    unique_alleles = np.unique(marker_data)
    return len(unique_alleles)

# Polymorphic Information Content (PIC)
def polymorphic_information_content(marker_data):
    unique, counts = np.unique(marker_data, return_counts=True)
    p_values = counts / len(marker_data)
    pic = 1 - sum(p_values ** 2)
    return pic

# Observed Heterozygosity (Obs_Het)
def observed_heterozygosity(marker_data):
    obs_het = np.sum(marker_data == 1) / len(marker_data)
    return obs_het

# Expected Heterozygosity (Exp_Het)
def expected_heterozygosity(marker_data):
    unique, counts = np.unique(marker_data, return_counts=True)
    p_values = counts / len(marker_data)
    exp_het = 1 - sum(p_values ** 2)
    return exp_het

# Gene Diversity (He)
def gene_diversity(marker_data):
    return expected_heterozygosity(marker_data)

# Shannon Index (I)
def shannon_index(marker_data):
    unique, counts = np.unique(marker_data, return_counts=True)
    proportions = counts / len(marker_data)
    shannon = -np.sum(proportions * np.log(proportions))
    return shannon

# Simpson’s Index (L)
def simpson_index(marker_data):
    unique, counts = np.unique(marker_data, return_counts=True)
    proportions = counts / len(marker_data)
    simpson = 1 - np.sum(proportions ** 2)
    return simpson

# ----------------------------------------------------------
# 🌍 Genetic Differentiation (Fst and Nm)
# ----------------------------------------------------------
def calculate_fst_nm(file_path):
    data = pd.read_excel(file_path)
    fst_nm_params = []

    for marker in data.columns[1:]:  # Skip first column (sample names)
        marker_data = data[marker].values

        # Calculate allele frequency
        allele_freqs = marker_data.mean() / 2

        # Variance calculations
        variance_total = np.var(marker_data)
        variance_between = allele_freqs * (1 - allele_freqs)

        # Fst and Nm
        fst_value = variance_between / variance_total if variance_total != 0 else np.nan
        nm_value = 0.25 * (1 - fst_value) / fst_value if fst_value not in [0, np.nan] else np.nan

        fst_nm_params.append({
            "Marker": marker,
            "Fst": fst_value,
            "Nm": nm_value
        })

    fst_nm_df = pd.DataFrame(fst_nm_params)
    return fst_nm_df

# ----------------------------------------------------------
# 🧮 Full SSR Data Analysis Pipeline
# ----------------------------------------------------------
def analyze_ssr_data(file_path):
    data = pd.read_excel(file_path)
    genetic_params = []

    for marker in data.columns[1:]:  # Skip first column (sample names)
        marker_data = data[marker].values
        genetic_params.append({
            "Marker": marker,
            "MAF": major_allele_frequency(marker_data),
            "Na": number_of_alleles(marker_data),
            "PIC": polymorphic_information_content(marker_data),
            "Obs_Het": observed_heterozygosity(marker_data),
            "Exp_Het": expected_heterozygosity(marker_data),
            "He": gene_diversity(marker_data),
            "Shannon_I": shannon_index(marker_data),
            "Simpson_L": simpson_index(marker_data)
        })

    genetic_params_df = pd.DataFrame(genetic_params)
    fst_nm_df = calculate_fst_nm(file_path)

    return genetic_params_df, fst_nm_df

# ----------------------------------------------------------
# 🚀 Main Function for Google Colab Execution
# ----------------------------------------------------------
def main():
    print("📁 Please upload your SSR genotype Excel file (.xlsx)")
    uploaded = files.upload()

    file_path = list(uploaded.keys())[0]

    print("\n🔄 Processing genetic diversity analysis...")
    genetic_analysis_result, fst_nm_result = analyze_ssr_data(file_path)

    print("\n✅ Genetic Diversity Analysis Results:")
    display(genetic_analysis_result)

    print("\n✅ Genetic Differentiation (Fst) and Gene Flow (Nm) Results:")
    display(fst_nm_result)

    # Save outputs as CSV files
    genetic_analysis_result.to_csv("genetic_analysis_result.csv", index=False)
    fst_nm_result.to_csv("fst_nm_result.csv", index=False)

    print("\n💾 Download your results:")
    files.download("genetic_analysis_result.csv")
    files.download("fst_nm_result.csv")

# ----------------------------------------------------------
# ▶️ Run the Analysis
# ----------------------------------------------------------
main()


📁 Please upload your SSR genotype Excel file (.xlsx)


IndexError: list index out of range

In [7]:
# ================================================
# 🌾 SSR Marker Full Analysis Pipeline (Colab Ready)
# ================================================

# 1️⃣ Install required packages
!pip install --quiet scikit-bio openpyxl xlsxwriter seaborn statsmodels

# 2️⃣ Import libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import MDS
from scipy.stats import chi2_contingency
from statsmodels.stats.multitest import multipletests
from skbio import DistanceMatrix
from skbio.tree import nj
from google.colab import files

# ------------------------------------------------
# 3️⃣ Upload SSR Excel or CSV file
# ------------------------------------------------
print("📤 Upload your SSR Excel or CSV file:")
uploaded = files.upload()
file_path = Path(list(uploaded.keys())[0])
print(f"✅ Uploaded file: {file_path}")

if file_path.suffix in [".xlsx", ".xls"]:
    df_raw = pd.read_excel(file_path)
else:
    df_raw = pd.read_csv(file_path)

# ------------------------------------------------
# 4️⃣ Preprocess: Detect sample IDs and binarize
# ------------------------------------------------
df = df_raw.copy()
if df.shape[1] >= 2 and (df.iloc[:, 0].dtype == object or df.iloc[:, 0].nunique() == df.shape[0]):
    df = df.set_index(df.columns[0])
else:
    df.index = [f"Sample_{i+1}" for i in range(df.shape[0])]

def to_binary(v):
    if pd.isna(v):
        return 0
    try:
        fv = float(v)
        return 1 if fv > 0 else 0
    except Exception:
        s = str(v).strip().lower()
        if s in {"1","yes","y","+","present","p","true","t"}:
            return 1
        if s in {"0","no","n","-","absent","a","false","f"}:
            return 0
        return 1

binary = df.applymap(to_binary).astype(int)
binary = binary.loc[(binary.sum(axis=1) > 0), (binary.sum(axis=0) > 0)]
print("✅ Cleaned binary SSR matrix (Samples × Markers):")
display(binary.head())

os.makedirs("results", exist_ok=True)
binary.to_csv("results/SSR_binary_matrix.csv")

# ------------------------------------------------
# 5️⃣ Jaccard Distance (samples)
# ------------------------------------------------
print("🔹 Computing Jaccard distance between samples...")
X = binary.values.astype(bool)
sample_labels = list(binary.index.astype(str))

condensed = pdist(X, metric="jaccard")
dist_mat = squareform(condensed)
dist_df = pd.DataFrame(dist_mat, index=sample_labels, columns=sample_labels)
dist_df.to_csv("results/Jaccard_distance_matrix.csv")

# ------------------------------------------------
# 6️⃣ UPGMA Tree + Save .newick
# ------------------------------------------------
Z_avg = linkage(condensed, method="average")

plt.figure(figsize=(10, 6))
dendrogram(Z_avg, labels=sample_labels, leaf_rotation=90)
plt.title("UPGMA Dendrogram (Sample-based)")
plt.ylabel("Jaccard Distance")
plt.tight_layout()
plt.savefig("results/UPGMA_dendrogram.png", dpi=300)
plt.close()

def linkage_to_newick(Z, labels):
    n = len(labels)
    nodes = {i: (labels[i], 0.0) for i in range(n)}
    next_id = n
    for (a, b, dist, _) in Z:
        a, b = int(a), int(b)
        name_a, h_a = nodes[a]
        name_b, h_b = nodes[b]
        h = dist / 2.0
        new_name = f"({name_a}:{max(h-h_a,0):.5f},{name_b}:{max(h-h_b,0):.5f})"
        nodes[next_id] = (new_name, h)
        next_id += 1
    return nodes[next_id-1][0] + ";"

upgma_newick = linkage_to_newick(Z_avg, sample_labels)
with open("results/UPGMA_tree.newick", "w") as f:
    f.write(upgma_newick)

# ------------------------------------------------
# 7️⃣ Neighbor-Joining (NJ) Tree + Save .newick
# ------------------------------------------------
dm = DistanceMatrix(dist_mat, ids=sample_labels)
nj_tree = nj(dm)

# Version-safe NJ tree writing
with open("results/NJ_tree.newick", "w") as f:
    nj_tree.write(f, format="newick")

# ------------------------------------------------
# 8️⃣ Marker Diversity Statistics
# ------------------------------------------------
freqs = binary.mean(axis=0)
marker_stats = pd.DataFrame({
    "Marker": binary.columns,
    "Presence_Count": binary.sum(axis=0),
    "Absence_Count": binary.shape[0] - binary.sum(axis=0),
    "Allele_Freq": freqs,
    "Shannon_Index": [-p*np.log2(p)-(1-p)*np.log2(1-p) if 0<p<1 else 0 for p in freqs],
    "Simpson_Index": [1-(p**2+(1-p)**2) for p in freqs],
    "PIC_approx": 2*freqs*(1-freqs)
}).set_index("Marker")
marker_stats.to_csv("results/Marker_statistics.csv")

# ------------------------------------------------
# 9️⃣ Linkage Disequilibrium (LD)
# ------------------------------------------------
results=[]
markers=list(binary.columns)
for i in range(len(markers)):
    for j in range(i+1,len(markers)):
        m1,m2=markers[i],markers[j]
        ctab=pd.crosstab(binary[m1],binary[m2])
        if ctab.shape!=(2,2): continue
        chi2,p,_,_=chi2_contingency(ctab,correction=False)
        results.append((m1,m2,chi2,p))
ld_df=pd.DataFrame(results,columns=["Marker1","Marker2","Chi2","Pvalue"])
if len(ld_df)>0:
    _, p_bon, _, _ = multipletests(ld_df["Pvalue"], method="bonferroni")
    ld_df["Pvalue_Bonferroni"] = p_bon
ld_df.to_csv("results/LD_results.csv",index=False)

# ------------------------------------------------
# 🔟 PCA & MDS (Sample-Based)
# ------------------------------------------------
Xs = StandardScaler().fit_transform(binary.fillna(0).values)
pca = PCA(n_components=2)
pcs = pca.fit_transform(Xs)
pca_df = pd.DataFrame(pcs, index=binary.index, columns=["PC1","PC2"])
plt.figure(figsize=(8,6))
plt.scatter(pca_df["PC1"], pca_df["PC2"], s=80)
for i,txt in enumerate(pca_df.index):
    plt.annotate(txt,(pca_df["PC1"].iat[i],pca_df["PC2"].iat[i]),fontsize=8)
plt.title("PCA of SSR Genotypes (Samples)")
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
plt.tight_layout()
plt.savefig("results/PCA_plot.png",dpi=300)
plt.close()

mds = MDS(n_components=2, dissimilarity="precomputed", random_state=42)
coords = mds.fit_transform(dist_mat)
plt.figure(figsize=(8,6))
plt.scatter(coords[:,0],coords[:,1],s=80)
for i,txt in enumerate(sample_labels):
    plt.annotate(txt,(coords[i,0],coords[i,1]),fontsize=8)
plt.title("MDS of Jaccard Distances (Samples)")
plt.xlabel("Dimension 1"); plt.ylabel("Dimension 2")
plt.tight_layout()
plt.savefig("results/MDS_plot.png",dpi=300)
plt.close()

# ------------------------------------------------
# 11️⃣ AMOVA-like Partition (Sample-Based)
# ------------------------------------------------
row_medians = np.median(dist_mat, axis=1)
median_all = np.median(dist_mat)
g1 = np.where(row_medians < median_all)[0]
g2 = np.where(row_medians >= median_all)[0]

def mean_upper(mat, idx):
    if len(idx)<=1: return np.nan
    sub = mat[np.ix_(idx,idx)]
    return np.nanmean(sub[np.triu_indices_from(sub,k=1)])

within1 = mean_upper(dist_mat,g1)
within2 = mean_upper(dist_mat,g2)
between = np.nanmean(dist_mat[np.ix_(g1,g2)])
total = np.nanmean(dist_mat[np.triu_indices_from(dist_mat,k=1)])
pct_between = between/total if total>0 else np.nan

amova_df=pd.DataFrame({
    "Group":["G1","G2","Between","Total"],
    "Mean_Dist":[within1,within2,between,total],
    "Pct_Between":[np.nan,np.nan,pct_between,np.nan]
})
amova_df.to_csv("results/AMOVA_summary.csv",index=False)

# ------------------------------------------------
# 12️⃣ Heatmap & Stacked Bar
# ------------------------------------------------
plt.figure(figsize=(12,6))
sns.heatmap(binary.T, cmap="coolwarm", cbar_kws={"label":"Allele Presence (0/1)"})
plt.title("SSR Marker Heatmap (Samples × Markers)")
plt.tight_layout()
plt.savefig("results/Heatmap.png",dpi=300)
plt.close()

prop = binary.apply(lambda x: x.value_counts(normalize=True), axis=0).T.fillna(0)
prop[[0,1]].plot(kind='bar', stacked=True, figsize=(12,6))
plt.title("Stacked Bar of SSR Markers")
plt.xlabel("Markers"); plt.ylabel("Proportion")
plt.legend(["Absent (0)","Present (1)"])
plt.tight_layout()
plt.savefig("results/Stacked_Bar.png",dpi=300)
plt.close()

# ------------------------------------------------
# 13️⃣ Save Everything to a Single Excel Workbook
# ------------------------------------------------
excel_out = "results/SSR_analysis_results.xlsx"
with pd.ExcelWriter(excel_out, engine="xlsxwriter") as writer:
    binary.to_excel(writer, sheet_name="Binary_Matrix")
    dist_df.to_excel(writer, sheet_name="Jaccard_Dist")
    marker_stats.to_excel(writer, sheet_name="Marker_Stats")
    ld_df.to_excel(writer, sheet_name="LD_Results", index=False)
    pca_df.to_excel(writer, sheet_name="PCA")
    amova_df.to_excel(writer, sheet_name="AMOVA")

print("✅ All analysis complete! Files saved in 'results/' folder.")
files.download(excel_out)
files.download("results/UPGMA_tree.newick")
files.download("results/NJ_tree.newick")


📤 Upload your SSR Excel or CSV file:


Saving Gel doc final data.xlsx to Gel doc final data.xlsx
✅ Uploaded file: Gel doc final data.xlsx
✅ Cleaned binary SSR matrix (Samples × Markers):


/tmp/ipython-input-2701329685.py:62: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  binary = df.applymap(to_binary).astype(int)


,AF8,AF10,AF11,AF13,AF16,AF18,AF25,AF43,AF48,AF53,AF54,AF55,AF64
Genotype,,,,,,,,,,,,,
ChAF1,1,0,1,1,0,1,1,1,0,1,1,1,1
ChAF2,1,1,1,1,0,1,1,1,0,1,1,1,1
ChAF3,1,1,1,1,0,1,1,1,0,1,1,1,1
ChAF4,1,0,1,1,0,0,1,1,0,1,1,1,1
ChAF5,1,1,1,1,1,1,1,1,0,1,1,1,1


🔹 Computing Jaccard distance between samples...
✅ All analysis complete! Files saved in 'results/' folder.


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
# --- 📊 Refined Heatmap and Network Graph for Journal Submission ---
# ✅ Works with both CSV and Excel files directly in Google Colab

# 📦 Install required packages
!pip install networkx matplotlib pandas openpyxl seaborn --quiet

# 📁 Step 1: Upload your file
from google.colab import files
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

print("📤 Please upload your Jaccard distance matrix file (.csv or .xlsx):")
uploaded = files.upload()
file_path = list(uploaded.keys())[0]  # take the first uploaded file
print(f"✅ File uploaded: {file_path}")

# --- Step 2: Read the matrix file automatically ---
if file_path.endswith(".csv"):
    df = pd.read_csv(file_path, index_col=0)
elif file_path.endswith((".xls", ".xlsx")):
    df = pd.read_excel(file_path, index_col=0)
else:
    raise ValueError("❌ Unsupported file format. Please upload a .csv or .xlsx file.")

print("\n📄 Data shape:", df.shape)
print("📊 Preview:")
display(df.head())

# --- Step 3: Convert Jaccard Distance → Similarity ---
# Distance: 0 = identical, 1 = completely different
# Similarity = 1 - distance
sim_df = 1 - df

# --- Step 4: Create and display Heatmap ---
heatmap_path = "jaccard_heatmap.png"
plt.figure(figsize=(14, 10))
sns.heatmap(sim_df, annot=False, cmap="YlGnBu", cbar_kws={'label': 'Jaccard Distance'},
            linewidths=0.5, linecolor='gray', xticklabels=df.index, yticklabels=df.index)

# Add title and axis labels
plt.title('Jaccard Distance Matrix of Entities', fontsize=16, weight='bold')
plt.xlabel('Sample ID', fontsize=12, weight='bold')
plt.ylabel('Sample ID', fontsize=12, weight='bold')

# Rotate the x-axis labels for better readability
plt.xticks(rotation=90, fontsize=10)
plt.yticks(rotation=0, fontsize=10)

# Save the heatmap image
plt.tight_layout()
plt.savefig(heatmap_path, dpi=300, bbox_inches="tight")
plt.close()  # Close the plot to free up memory
print(f"✅ Heatmap image saved as {heatmap_path}")

# --- Step 5: Convert similarity matrix to edge list ---
edges = sim_df.stack().reset_index()
edges.columns = ["u", "v", "similarity"]

# Remove self-loops and duplicate symmetric pairs
edges = edges[edges["u"] != edges["v"]]
edges = edges.drop_duplicates(subset=["u", "v"])

# --- Step 6: Filter by similarity threshold ---
threshold = 0.5  # 👈 Adjust this value (0.0–1.0)
edges_filtered = edges[edges["similarity"] > threshold]
print(f"📉 Edges retained with similarity > {threshold}: {edges_filtered.shape[0]}")

# --- Step 7: Build network graph ---
G = nx.Graph()
for _, row in edges_filtered.iterrows():
    G.add_edge(str(row["u"]), str(row["v"]), weight=float(row["similarity"]))

print("\n📦 Graph Summary:")
print(f"   • Nodes: {G.number_of_nodes()}")
print(f"   • Edges: {G.number_of_edges()}")

if G.number_of_nodes() == 0:
    raise ValueError("❌ No edges above threshold. Try lowering the threshold value.")

# --- Step 8: Prepare visualization parameters ---
deg = dict(G.degree())
maxdeg = max(deg.values()) if deg else 1
node_sizes = [600 + (deg[n] / maxdeg) * 1400 for n in G.nodes()]

weights = nx.get_edge_attributes(G, "weight")
if len(weights) > 0:
    maxw = max(weights.values())
    edge_widths = [max(0.6, (w / maxw) * 6) for w in weights.values()]
else:
    edge_widths = [1 for _ in G.edges()]

# Layout options: spring_layout, kamada_kawai_layout, circular_layout, etc.
pos = nx.spring_layout(G, seed=42, k=0.4, iterations=300)

# --- Step 9: Draw the network graph ---
network_graph_path = "jaccard_network_refined.png"
plt.figure(figsize=(14, 14))

# Refined node and edge drawing
nx.draw_networkx_nodes(
    G, pos,
    node_size=node_sizes,
    node_color="skyblue",  # Soft color for nodes
    edgecolors="black",  # Black edges around nodes
    linewidths=1.5  # Thicker borders around nodes
)

# Refined labels with bold font
nx.draw_networkx_labels(G, pos, font_size=12, font_weight="bold", font_color="black")

# Refined edges with width proportional to similarity
nx.draw_networkx_edges(G, pos, width=edge_widths, alpha=0.7, edge_color="gray")

# Edge labels with similarity values
edge_labels = {(u, v): f"{d['weight']:.2f}" for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7, font_weight="bold")

# Title and layout adjustments for journal style
plt.title("📊 Genotype Network Based on Jaccard Similarity", fontsize=18, fontweight='bold')
plt.axis("off")
plt.tight_layout()

# Save the refined network graph image
plt.savefig(network_graph_path, dpi=600, bbox_inches="tight")
plt.close()  # Close the plot to free up memory
print(f"✅ Refined network graph image saved as {network_graph_path}")

# --- Step 10: Download the refined network graph and heatmap ---
from google.colab import files
files.download(heatmap_path)
files.download(network_graph_path)


📤 Please upload your Jaccard distance matrix file (.csv or .xlsx):


Saving Jaccard_distance_matrix.csv to Jaccard_distance_matrix (1).csv
✅ File uploaded: Jaccard_distance_matrix (1).csv

📄 Data shape: (38, 38)
📊 Preview:


,ChAF1,ChAF2,ChAF3,ChAF4,ChAF5,ChAF6,ChAF7,ChAF8,ChAF9,ChAF10,...,MaAf9,MaAf10,MaAf11,MaAf12,MaAf13,MaAf14,MaAf15,MaAf16,MaAf17,MaAf18
ChAF1,0.000000,0.090909,0.090909,0.100000,0.166667,0.181818,0.166667,0.166667,0.166667,0.250000,...,0.384615,0.416667,0.333333,0.272727,0.461538,0.384615,0.461538,0.250000,0.416667,0.500000
ChAF2,0.090909,0.000000,0.000000,0.181818,0.083333,0.090909,0.083333,0.083333,0.083333,0.166667,...,0.307692,0.333333,0.250000,0.181818,0.384615,0.307692,0.384615,0.166667,0.333333,0.416667
ChAF3,0.090909,0.000000,0.000000,0.181818,0.083333,0.090909,0.083333,0.083333,0.083333,0.166667,...,0.307692,0.333333,0.250000,0.181818,0.384615,0.307692,0.384615,0.166667,0.333333,0.416667
ChAF4,0.100000,0.181818,0.181818,0.000000,0.250000,0.272727,0.250000,0.250000,0.250000,0.333333,...,0.333333,0.500000,0.416667,0.363636,0.416667,0.461538,0.416667,0.181818,0.363636,0.454545
ChAF5,0.166667,0.083333,0.083333,0.250000,0.000000,0.166667,0.000000,0.000000,0.000000,0.083333,...,0.230769,0.250000,0.166667,0.250000,0.307692,0.230769,0.307692,0.083333,0.250000,0.333333


✅ Heatmap image saved as jaccard_heatmap.png
📉 Edges retained with similarity > 0.5: 1400

📦 Graph Summary:
   • Nodes: 38
   • Edges: 700


/tmp/ipython-input-3008049792.py:123: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-3008049792.py:126: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  plt.savefig(network_graph_path, dpi=600, bbox_inches="tight")


✅ Refined network graph image saved as jaccard_network_refined.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 📦 Install necessary packages (run once)
!pip install networkx matplotlib pandas --quiet

# 📁 Step 1: Upload your CSV or Excel file
from google.colab import files
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Upload the file
uploaded = files.upload()
csv_path = list(uploaded.keys())[0]  # Take the first uploaded file
print("✅ File uploaded:", csv_path)

# Step 2: Read the CSV/Excel file
df = pd.read_csv(csv_path)  # Change to pd.read_excel if your file is Excel
print("\n📄 File Shape:", df.shape)
print("📊 Columns:", list(df.columns))
display(df.head(10))

# --- Step 3: Infer Edge List from the Dataset ---
def infer_edges_from_df(df):
    """
    This function infers the edge list from various formats in the dataset.
    It checks for different structures such as an edge list, adjacency matrix, or a simple 2-column edge list.
    """
    # Identify object and numeric columns
    obj_cols = [c for c in df.columns if df[c].dtype == "object" or df[c].dtype.name == "string"]
    num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]

    # Case 1: Edge list (node1, node2, weight)
    if len(obj_cols) >= 2 and len(num_cols) >= 1:
        node1, node2, weight = obj_cols[0], obj_cols[1], num_cols[0]
        edges = df[[node1, node2, weight]].dropna().rename(columns={node1:"u", node2:"v", weight:"w"})
        return edges[["u","v","w"]]

    # Case 2: Two columns → unweighted edge list
    if df.shape[1] == 2:
        ucol, vcol = df.columns[0], df.columns[1]
        edges = df.rename(columns={ucol:"u", vcol:"v"})
        edges["w"] = 1.0  # Assign weight 1.0 to all edges
        return edges[["u","v","w"]]

    # Case 3: Adjacency matrix
    if df.shape[0] == df.shape[1] or (pd.api.types.is_string_dtype(df.iloc[:,0]) and df.shape[1] > 2):
        if not pd.api.types.is_numeric_dtype(df.iloc[:,0]):
            df2 = df.set_index(df.columns[0])
            df_num = df2.apply(pd.to_numeric, errors="coerce")
            stacked = df_num.stack().reset_index()
            stacked.columns = ["u","v","w"]
            stacked = stacked.dropna(subset=["w"])
            stacked = stacked[stacked["w"] != 0]
            return stacked

    # Fallback: assume first three columns are u, v, w
    if df.shape[1] >= 3:
        edges = df.iloc[:, :3].rename(columns={df.columns[0]:"u", df.columns[1]:"v", df.columns[2]:"w"})
        return edges[["u","v","w"]].dropna()

    raise ValueError("❌ Could not infer edge list format. Please check your CSV columns.")

# Get edges from dataframe
edges = infer_edges_from_df(df)
print("\n✅ Inferred edges:", edges.shape[0])
display(edges.head(20))

# --- Step 4: Filter Edges Based on Weight Threshold ---
threshold = 0.0  # 👈 you can change this to 0.3, 0.5, etc. to filter weak edges
edges_filtered = edges[edges["w"] > threshold].copy()
print(f"📉 Filtered edges > {threshold}: {edges_filtered.shape[0]}")

# --- Step 5: Build the Graph ---
G = nx.Graph()  # Create an undirected graph
for _, row in edges_filtered.iterrows():
    u, v, w = str(row["u"]), str(row["v"]), float(row["w"])
    if u == v:  # Avoid self-loops
        continue
    G.add_edge(u, v, weight=w)

print("📦 Graph summary:")
print("   Nodes:", G.number_of_nodes())
print("   Edges:", G.number_of_edges())

# --- Step 6: Prepare Visualization ---
# Calculate node sizes based on degree (larger nodes have higher degrees)
deg = dict(G.degree())
maxdeg = max(deg.values()) if deg else 1
node_sizes = [600 + (deg[n] / maxdeg) * 1400 for n in G.nodes()]

# Calculate edge widths based on weight (thicker edges for stronger weights)
weights = nx.get_edge_attributes(G, "weight")
if len(weights) > 0:
    maxw = max(weights.values())
    edge_widths = [max(0.6, (w / maxw) * 6) for w in weights.values()]
else:
    edge_widths = [1 for _ in G.edges()]

# Set positions using spring layout
pos = nx.spring_layout(G, seed=42, k=0.4, iterations=200)

# --- Step 7: Draw the Network ---
plt.figure(figsize=(12, 12))

# Draw nodes
nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color="skyblue", edgecolors="black", linewidths=0.8)

# Draw node labels (optional)
nx.draw_networkx_labels(G, pos, font_size=9, font_weight="bold")

# Draw edges
nx.draw_networkx_edges(G, pos, width=edge_widths, alpha=0.9)

# Draw edge labels (optional)
edge_labels = {(u,v): f"{d['weight']:.2f}" for u,v,d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7)

# Clean up visualization
plt.axis("off")
plt.title("📊 Network Plot from LD Results", fontsize=16)
plt.tight_layout()

# --- Step 8: Save and Display the Figure ---
out_path = "network_plot.png"
plt.savefig(out_path, dpi=300, bbox_inches="tight")  # Save the figure
plt.show()  # Display the figure

print("\n✅ Saved figure as:", out_path)

# 📥 Step 9: Download the figure and results
from google.colab import files
files.download(out_path)  # Download the network plot image


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Data setup
data = {
    "Marker1": ['tub1', 'tub1', 'tub1', 'aflR', 'aflR', 'OmtB'],
    "Marker2": ['aflR', 'OmtB', 'Omt1', 'OmtB', 'Omt1', 'Omt1'],
    "Chi2": [0.057057057, 5.147089947, 3.619047619, 0.088030888, 0.599099099, 5.583673469],
    "Pvalue": [0.811209343, 0.023285365, 0.057121564, 0.76669556, 0.438921967, 0.018128658],
    "Pvalue_Bonferroni": [1, 0.139712191, 0.342729384, 1, 1, 0.10877195]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Adjusted significance level after Bonferroni correction
alpha_bonferroni = 0.05 / len(df)

# Create a new column to check if the p-values are significant (before and after Bonferroni correction)
df['Significant (p-value < 0.05)'] = df['Pvalue'] < 0.05
df['Significant (Bonferroni)'] = df['Pvalue_Bonferroni'] < alpha_bonferroni

# Calculate Cramér's V for each comparison (effect size)
# Cramér's V = sqrt(Chi2 / (n * (min(k1, k2) - 1)))
# where n = total sample size, k1 and k2 are the number of categories for each variable
n = 100  # Placeholder for total sample size
k1 = 2   # Assuming 2 categories for each marker (simplification)
k2 = 2   # Same assumption for simplicity

df['Cramer\'s V'] = np.sqrt(df['Chi2'] / (n * (min(k1, k2) - 1)))

# Create a heatmap for the significance of the p-values
plt.figure(figsize=(10, 6))
sns.heatmap(df[['Chi2', 'Pvalue', 'Pvalue_Bonferroni']].T,
            annot=True, fmt='.3f', cmap='coolwarm', cbar_kws={'label': 'Values'},
            linewidths=0.5)
plt.title("Heatmap of Chi2, P-value and Bonferroni-corrected P-value", fontsize=14)
plt.show()

# Visualize the significance results using bar plots for better understanding
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Bar plot for p-value significance
axes[0].barh(df['Marker1'] + ' vs ' + df['Marker2'], df['Pvalue'], color=df['Significant (p-value < 0.05)'].map({True: 'green', False: 'red'}))
axes[0].set_title("Significance based on p-value < 0.05")
axes[0].set_xlabel('P-value')
axes[0].set_ylabel('Marker Comparisons')

# Bar plot for Bonferroni-corrected p-value significance
axes[1].barh(df['Marker1'] + ' vs ' + df['Marker2'], df['Pvalue_Bonferroni'], color=df['Significant (Bonferroni)'].map({True: 'green', False: 'red'}))
axes[1].set_title("Significance based on Bonferroni-corrected p-value")
axes[1].set_xlabel('Bonferroni-corrected P-value')

plt.tight_layout()
plt.show()

# Scatter plot for Chi2 vs P-value for visualization of strength and significance
plt.figure(figsize=(8, 6))
plt.scatter(df['Chi2'], df['Pvalue'], c=df['Significant (p-value < 0.05)'].map({True: 'green', False: 'red'}),
            label='Significant', s=100, alpha=0.7)
plt.xlabel('Chi2')
plt.ylabel('P-value')
plt.title('Scatter Plot of Chi2 vs P-value')
plt.show()

# Scatter plot for Chi2 vs Cramér's V (to visualize the effect size)
plt.figure(figsize=(8, 6))
plt.scatter(df['Chi2'], df['Cramer\'s V'], c=df['Significant (p-value < 0.05)'].map({True: 'green', False: 'red'}),
            label='Significant', s=100, alpha=0.7)
plt.xlabel('Chi2')
plt.ylabel("Cramér's V (Effect Size)")
plt.title("Chi2 vs Cramér's V (Effect Size)")
plt.show()
